En este código se quiere realizar una implementación de un simulador de físicas sencillo.

In [ ]:
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
import sys
import numpy as np
import time as t
import random as r

#Tamaño de la ventana
width = 800
height = 800

xmin=-float(width/2)
xmax=float(width/2)
ymin=-float(height/2)
ymax=float(height/2)

radius = 30.0 #pix

v_coef = 30

g = np.array([0.0,-0.6])
v = np.array([(r.random()*v_coef)*(1-1/2),
              (r.random()*(v_coef/2))])
pos = []

VEL_MAX = 30

def init():
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glMatrixMode(GL_PROJECTION)
    gluOrtho2D(xmin, xmax, ymin, ymax)
    glMatrixMode(GL_MODELVIEW)


def actualizar():
    global v,g,pos
    
    if pos != []:
        t.sleep(0.009)
        new_v = np.add(v,g) #actualiza velocidad

        if not mayor_que_vel_limite(new_v): #limitamos la velocidad
            v = new_v    

        new_pos = np.add(pos,v)

        pos = out_screen(new_pos)

        #print(pos)

        glutPostRedisplay()

    
# Process new_pos and checks is on screen
def out_screen(new_pos):
    global v
    if new_pos[0] > (xmax - radius) or new_pos[0] < (xmin + radius):
        v = np.multiply(v,[-0.75,0.97])
        
        if new_pos[0] > (xmax - radius): # DERECHA
            new_pos = np.add(new_pos,[(xmax - radius)-new_pos[0],0])
          
        if new_pos[0] < (xmin + radius): # IZQUIERDA
            new_pos = np.add(new_pos,[(xmin + radius)-new_pos[0],0])
        
    if new_pos[1] > (ymax - radius) or new_pos[1] < (ymin + radius):
        v = np.multiply(v,[0.97,-0.75])
        
        #if new_pos[1] > (ymax - radius): # ARRIBA
         #  new_pos = np.add(new_pos,[0,(ymax-radius)-new_pos[1]])          
            
        if new_pos[1] < (ymin + radius): # ABAJO
            new_pos = np.add(new_pos,[0,(ymin+radius)-new_pos[1]])
        
    return new_pos
    
    
def mayor_que_vel_limite(v):
    return (np.sqrt(np.power(v[0],2) + np.power(v[1],2))) > VEL_MAX
        
def mouse(button, state, x, y):
    global pos
    
    realPosMousex = ((xmax-xmin)/width*x+xmin)
    realPosMousey = ((ymin-ymax)/height*y+ymax)
    
    if button == GLUT_LEFT_BUTTON and state==GLUT_DOWN:
        pos = np.array([realPosMousex,realPosMousey])

    if button == GLUT_RIGHT_BUTTON and state==GLUT_DOWN:
        pos = []

def dibujar_circulo():
    
    glClearColor(0.0, 0.0, 0.0,1.0);
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT);
    if pos != []:

        glBegin(GL_TRIANGLE_FAN);

        glColor3f(1,1,1);

        glVertex2fv(pos);

        alfa = 0;
        for i in range(0,361):
            glVertex2f(pos[0] + radius * np.cos(alfa), pos[1] + radius * np.sin(alfa));
            alfa += np.pi / 180; # 1 point each degree (in radians)

        glEnd()

    glFlush()

    
    
glutInit(sys.argv)

glutInitDisplayMode(GLUT_SINGLE | GLUT_RGB)
# Atributos de la ventana
glutInitWindowPosition(950,200)
glutInitWindowSize(width,height)
glutCreateWindow("Particulas")

init()

glutDisplayFunc(dibujar_circulo)
glutMouseFunc(mouse)
glutIdleFunc(actualizar)
#glutPassiveMotionFunc(cursor)
glutMainLoop()

<ipython-input-1-8290c0a87f0c>:39: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if pos != []:
<ipython-input-1-8290c0a87f0c>:98: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if pos != []:
